# 모델링

In [1]:
import pandas as pd

# 데이터셋 로드
train = pd.read_csv('채무_불이행_예측train.csv')
test = pd.read_csv('채무_불이행_예측test.csv')

# 데이터 전처리
# 'train'과 'test' 데이터에서 'UID' 열을 제거하고, 'test'의 'UID'는 별도로 저장
train = train.drop('UID', axis = 1)
UID = test.pop('UID')

# 의미 없는 변수 제거
train = train.drop(['신용 거래 연수', '최대 신용한도', '신용 문제 발생 횟수', '마지막 연체 이후 경과 개월 수', '대출 목적', '현재 대출 잔액', '현재 미상환 신용액'], axis = 1)
test = test.drop(['신용 거래 연수', '최대 신용한도', '신용 문제 발생 횟수', '마지막 연체 이후 경과 개월 수', '대출 목적', '현재 대출 잔액', '현재 미상환 신용액'], axis = 1)

# 타겟 변수 저장
target = train.pop('채무 불이행 여부')

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = train.columns[train.dtypes == object]

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

# 모델 학습
# CatBoost
from catboost import CatBoostClassifier
cat = CatBoostClassifier(verbose = 0, random_state = 0)
cat.fit(train, target)

# 제출
pred = cat.predict_proba(test)
submit = pd.DataFrame({'UID':UID,
                    '채무 불이행 확률':pred[:, 1]})
submit.to_csv('채무_불이행_예측sample_submission.csv', index = False)
